In [2]:
def plot_timeseries(df, variables_array, start_date, end_date, ylabel='', days_interval=0,
                   minutes_interval=180, save_figure_name = '', rolling_time=0, 
                   reference_line=[], figure_size=[30,10], ticks=[], legend=[], special_dates={}):
    
    start_date = pd.to_datetime(start_date,format="%d/%m/%Y %H:%M")
    end_date = pd.to_datetime(end_date,format="%d/%m/%Y %H:%M")

    df_filtered = df[ (df.index > start_date) & (df.index < end_date) ];
    
    print(df_filtered.shape)

    if(days_interval==0):
        days_interval = int((df.index[-1]-df.index[0]).days/30)
        print(days_interval)
    rule = rrulewrapper(DAILY, interval=days_interval)
    minor_rule = rrulewrapper(MINUTELY, interval=minutes_interval)

    loc = RRuleLocator(rule)
    formatter = DateFormatter('%d-%m-%Y')
    minor_loc = RRuleLocator(minor_rule)
    minor_formatter = DateFormatter('%H:%M')

    fig, ax = plt.subplots();
    xs = {}
    series = {}
    smask = {}
    df_return = pd.DataFrame()

    for variable in variables_array:
        plt.plot_date(df_filtered.index, df_filtered[variable], linestyle='-', marker='o');
        plt.title(variables_array[0])
            
        if(rolling_time>0):
            plt.plot_date(df_filtered.index, df_filtered[variable].rolling(rolling_time).mean(), linestyle='-', marker='o');
      
    if(len(reference_line)>0):
        for value in reference_line:
            ax.axhline(value, color='black', lw=2)

    if(len(ticks)>2):
        ax.set_yticks(np.arange(ticks[0], ticks[1], ticks[2]))
        
    if(len(legend)>0):
        ax.legend(legend)
    
    ymin, ymax = ax.get_ylim()
    for date in special_dates:
        d = pd.to_datetime(date , format="%d/%m/%Y")
        if( (d>=start_date) and (d <= end_date )):
            plt.axvspan(d, d+pd.Timedelta('5 days'), color='grey', alpha=0.5)
            position = uniform(0,1)
            plt.text(d,ymin+position*(ymax-ymin)/2,special_dates[date],fontsize=14)
         
            
    if(days_interval<=2):
        ax.xaxis.set_major_locator(loc);
        ax.xaxis.set_major_formatter(formatter); 
        ax.xaxis.set_tick_params(which='major', rotation=90, labelsize=10, pad=60)
        
        ax.xaxis.set_minor_locator(minor_loc);
        ax.xaxis.set_minor_formatter(minor_formatter);
        ax.xaxis.set_tick_params(which='minor', labelsize=10, rotation=90)
    else:
        ax.xaxis.set_major_locator(loc);
        ax.xaxis.set_major_formatter(formatter); 
        ax.xaxis.set_tick_params(which='major', rotation=90, labelsize=10, pad=0)


    fig.set_figheight(figure_size[1])
    fig.set_figwidth(figure_size[0])
    plt.grid(True)
    
    if(len(ylabel)>0):
        plt.ylabel(ylabel)
    if(len(save_figure_name)>0):
        plt.savefig(save_figure_name, dpi=500)
    plt.show()

    return df_filtered